<a href="https://colab.research.google.com/github/NextWaveGit/deeplearning/blob/master/yelpreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,CuDNNLSTM
from keras.layers.embeddings import Embedding

import keras.layers as layers
from keras.engine import Layer
import tensorflow_hub as hub
from keras import backend as K
import tensorflow as tf
from keras.models import Model, load_model

import time

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
#from sklearn.manifold import TSNE

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/"

In [0]:
file = '/content/drive/My Drive/ml/yelp_review.csv'
df = pd.read_csv(file, usecols = ['stars', 'text'], error_bad_lines=False,nrows=50000)

In [0]:
df= df.dropna()
#df = df[df.stars.apply(lambda x: x.isnumeric())]
df = df[df.stars.apply(lambda x: x !="")]
df = df[df.text.apply(lambda x: x !="")]

In [0]:
df.describe()

,stars
count,50000.000000
mean,3.731540
std,1.386127
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [0]:
df.head()

,stars,text
0,5,Super simple place but amazing nonetheless. It...
1,5,Small unassuming place that changes their menu...
2,5,Lester's is located in a beautiful neighborhoo...
3,4,Love coming here. Yes the place always needs t...
4,4,Had their chocolate almond croissant and it wa...


In [0]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

In [10]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
def clean_text(text):
     
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [22]:
start_time = time.time()
df['text'] = df['text'].map(lambda x: clean_text(x))
print("--- clean %s seconds ---" % (time.time() - start_time))

--- clean 70.80327939987183 seconds ---


In [0]:
df.head(10)

,stars,text
0,5,super simpl place amaz nonetheless around sinc...
1,5,small unassum place chang menu everi often coo...
2,5,lester locat beauti neighborhood sinc 1951 kno...
3,4,love come here yes place alway need floor swep...
4,4,chocol almond croissant amaz ! light butteri c...
5,5,cycl pub las vega blast ! got groupon rent bik...
6,4,would guess would abl get fair decent vietname...
7,4,alway drove past coffe hous wonder it final ma...
8,3,bad ! ! love gluten - free vegan version chees...
9,5,love place ! + + peggi great dog great job ! p...


In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])

sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [0]:
print(data.shape)

(50000, 50)


In [0]:
print(data[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   157   553     2    76  2340   104
   120   575    88   125    55   162   590 10055  3557   150  1566    37
    92    17]


In [0]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size, 100, input_length=50))
model_lstm.add(CuDNNLSTM(100))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
start_time = time.time()
model_lstm.fit(data, np.array(labels), validation_split=0.4, batch_size=128, epochs=10)
print("--- LSTM train %s seconds ---" % (time.time() - start_time))

Train on 30000 samples, validate on 20000 samples
Epoch 1/10
30000/30000 [==============================] - 11s 355us/step - loss: 0.4239 - acc: 0.8036 - val_loss: 0.3370 - val_acc: 0.8594
Epoch 2/10
30000/30000 [==============================] - 5s 179us/step - loss: 0.2880 - acc: 0.8809 - val_loss: 0.3352 - val_acc: 0.8563
Epoch 3/10
30000/30000 [==============================] - 5s 176us/step - loss: 0.2272 - acc: 0.9105 - val_loss: 0.3654 - val_acc: 0.8478
Epoch 4/10
30000/30000 [==============================] - 5s 175us/step - loss: 0.1770 - acc: 0.9313 - val_loss: 0.4698 - val_acc: 0.8428
Epoch 5/10
30000/30000 [==============================] - 5s 174us/step - loss: 0.1417 - acc: 0.9470 - val_loss: 0.4704 - val_acc: 0.8344
Epoch 6/10
30000/30000 [==============================] - 5s 176us/step - loss: 0.1115 - acc: 0.9584 - val_loss: 0.5527 - val_acc: 0.8276
Epoch 7/10
30000/30000 [==============================] - 5s 172us/step - loss: 0.0942 - acc: 0.9666 - val_loss: 0.5893 -

In [0]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    #model_conv.add(LSTM(100))
    model_conv.add(Flatten())
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [0]:
start_time = time.time()
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.4, batch_size=128, epochs = 10)
print("--- CNN train %s seconds ---" % (time.time() - start_time))

Train on 30000 samples, validate on 20000 samples
Epoch 1/10
30000/30000 [==============================] - 5s 168us/step - loss: 0.4825 - acc: 0.7601 - val_loss: 0.3539 - val_acc: 0.8484
Epoch 2/10
30000/30000 [==============================] - 3s 104us/step - loss: 0.3107 - acc: 0.8696 - val_loss: 0.3428 - val_acc: 0.8538
Epoch 3/10
30000/30000 [==============================] - 3s 103us/step - loss: 0.2382 - acc: 0.9063 - val_loss: 0.3656 - val_acc: 0.8465
Epoch 4/10
30000/30000 [==============================] - 3s 103us/step - loss: 0.1658 - acc: 0.9394 - val_loss: 0.4110 - val_acc: 0.8397
Epoch 5/10
30000/30000 [==============================] - 3s 103us/step - loss: 0.0969 - acc: 0.9700 - val_loss: 0.4838 - val_acc: 0.8337
Epoch 6/10
30000/30000 [==============================] - 3s 104us/step - loss: 0.0520 - acc: 0.9862 - val_loss: 0.5697 - val_acc: 0.8281
Epoch 7/10
30000/30000 [==============================] - 3s 104us/step - loss: 0.0276 - acc: 0.9938 - val_loss: 0.6571 - 

In [0]:
embeddings_index = dict()
f = open('/content/drive/My Drive/ml/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [0]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
#model_glove.add(LSTM(100))
model_glove.add(Flatten())
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
start_time = time.time()
model_glove.fit(data, np.array(labels), validation_split=0.4, batch_size = 128, epochs = 10)
print("--- glove train %s seconds ---" % (time.time() - start_time))

Train on 30000 samples, validate on 20000 samples
Epoch 1/10
30000/30000 [==============================] - 4s 138us/step - loss: 0.5856 - acc: 0.6924 - val_loss: 0.5279 - val_acc: 0.7378
Epoch 2/10
30000/30000 [==============================] - 2s 75us/step - loss: 0.5149 - acc: 0.7435 - val_loss: 0.4934 - val_acc: 0.7580
Epoch 3/10
30000/30000 [==============================] - 2s 75us/step - loss: 0.4850 - acc: 0.7659 - val_loss: 0.4778 - val_acc: 0.7696
Epoch 4/10
30000/30000 [==============================] - 2s 75us/step - loss: 0.4591 - acc: 0.7828 - val_loss: 0.4771 - val_acc: 0.7681
Epoch 5/10
30000/30000 [==============================] - 2s 76us/step - loss: 0.4383 - acc: 0.7969 - val_loss: 0.5057 - val_acc: 0.7546
Epoch 6/10
30000/30000 [==============================] - 2s 76us/step - loss: 0.4164 - acc: 0.8077 - val_loss: 0.4642 - val_acc: 0.7788
Epoch 7/10
30000/30000 [==============================] - 2s 75us/step - loss: 0.3972 - acc: 0.8207 - val_loss: 0.4629 - val_ac

In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [0]:
# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(1, activation='sigmoid')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  
  return model

In [0]:
seed = 7

# Create datasets (Only take up to 150 words for memory)
texts = df['text'].tolist()
texts = [' '.join(t.split()[0:50]) for t in texts]
texts = np.array(texts, dtype=object)[:, np.newaxis]


#X_train, X_test, y_train, y_test = train_test_split(texts, labeles, test_size=0.4, random_state=seed)


In [26]:
# Build and fit
model_elmo = build_model()
model_elmo.fit(texts, 
          np.array(labels),
          validation_split=0.4, batch_size = 128, epochs = 10)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_2 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________
Train on 30000 samples, validate on 20000 samples
Epoch 1/10
30000/30000 [==============================] - 488s 16ms/step - loss: 0.4950 - acc: 0.7521 - val_loss: 0.4415 - val_ac

In [0]:
# Build and fit
model_elmo = build_model()
model_elmo.fit(texts, 
          np.array(labels),
          validation_split=0.4, batch_size = 32, epochs = 10)